In [1]:
# import lsst.summit.extras

In [2]:
# lsst.summit.__file__

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
#from lsst.summit.extras.plotting.psfPlotting import makeAzElPlot, makeTableFromSourceCatalogs, makeFocalPlanePlot, makeEquatorialPlot
from MYpsfPlotting import makeAzElPlot, makeTableFromSourceCatalogs, makeFocalPlanePlot, makeEquatorialPlot, makeTableFromGrid
from lsst.afw import cameraGeom
from lsst.obs.lsst import LsstComCam, LsstCam
from lsst.geom import Point2D
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [3]:
from lsst.daf.butler import Butler

In [4]:
def pixel_to_camera(x, y, det):
    """
    Parameters
    ----------
    x, y : array
        Pixel coordinates.
    det : lsst.afw.cameraGeom.Detector
        Detector of interest.
    Returns
    -------
    cam_x, cam_y : array
        Focal plane position in millimeters in DVCS
        See https://lse-349.lsst.io/
    """
    tx = det.getTransform(cameraGeom.PIXELS, cameraGeom.FOCAL_PLANE)
    cam_x, cam_y = tx.getMapping().applyForward(np.vstack((x, y)))
    return cam_x.ravel(), cam_y.ravel()

Stuff to load simulations

In [5]:
import importlib.util
import sys
# sys.path.append("../../notebooks/")
# sys.path.append("/sdf/data/rubin/shared/image_quality/imsim/notebooks/")                #file with convertion between seqnum and collection
# from .sdf.data.rubin.shared.image_quality.imsim.notebooks.collection_dictionary_shared import collection_dictionary

spec = importlib.util.spec_from_file_location("collection_dictionary_shared", 
                                              "/sdf/data/rubin/shared/image_quality/imsim/notebooks/collection_dictionary_shared.py")
collection_dictionary = importlib.util.module_from_spec(spec)
sys.modules["module.name"] = collection_dictionary
spec.loader.exec_module(collection_dictionary)

collection_dictionary = collection_dictionary.collection_dictionary


In [6]:
folder = '/sdf/data/rubin/shared/image_quality/imsim/'
#Define the butler data configuration and collection (una tantum )
config = folder+'repo'
seqnum_base = 5023071800000
collection_dict = collection_dictionary()

In [7]:
seqnum = 145
visitid_complete = seqnum+seqnum_base
collections = collection_dict[visitid_complete]
collections

'pipetask_output/20241112T145223Z'

In [8]:
collection_dict[visitid_complete]

'pipetask_output/20241112T145223Z'

In [9]:
butler = Butler(config, collections=collections)

In [10]:
# butler.collections.query_info(collections[0], include_summary=True)

In [11]:
registry = butler.registry

In [12]:
postisr = list(registry.queryDatasets('icSrc', collections=collections, detector=90))
postisr
#exposure_ids = np.array([data.dataId['exposure'] for data in postisr])
# exposure_ids

[DatasetRef(DatasetType('icSrc', {band, instrument, detector, physical_filter, visit}, SourceCatalog), {instrument: 'LSSTCam', detector: 90, visit: 5023071800145, band: 'r', physical_filter: 'r_57'}, run='pipetask_output/20241112T145223Z', id=d33da782-559c-4844-8ea2-2391d1644e1f)]

In [16]:
# SULLE SORGENTI

icSrcs = {}
visit_id=postisr[0].dataId['visit']

assert False

for i in range(189):
    try:
        print(i)
        icSrcs[i] = butler.get("icSrc", dataId={"visit":visit_id, "detector":i})
    except:
        continue
        
visitInfo = butler.get("postISRCCD.visitInfo", dataId={"exposure":visit_id, "detector":90})
table = makeTableFromSourceCatalogs(icSrcs, visitInfo)

AssertionError: 

In [15]:
# SU GRIGLIA

calexps = {}
psfs = {}
table_grids= []
visit_id=postisr[0].dataId['visit']
n_grid = 10

visitInfo = butler.get("postISRCCD.visitInfo", dataId={"exposure":visit_id, "detector":90})
calexp = butler.get('calexp', dataId={"visit":visit_id, "detector":90})
psf = calexp.getPsf()
table_grid = makeTableFromGrid(n_grid, calexp, psf, visitInfo)
table_grids.append(table_grid)

for i in range(90, 99, 1):
    print(i)
    visitInfo = butler.get("postISRCCD.visitInfo", dataId={"exposure":visit_id, "detector":i})
    calexp = butler.get('calexp', dataId={"visit":visit_id, "detector":i})
    psf = calexp.getPsf()
    table_grid = makeTableFromGrid(n_grid, calexp, psf, visitInfo)
    table_grids.append(table_grid)
    del calexp

print(table_grids)

sss2
sss3
90
sss2
sss3
91
sss2
sss3
92
sss2
sss3
93
sss2
sss3
94
sss2
sss3
95
sss2
sss3
96
sss2
sss3
97
sss2
sss3
98
sss2
sss3
[<Table length=100>
       Ixx                 Ixy         ...        nw_x               nw_y       
     float64             float64       ...     float64[1]         float64[1]    
------------------ ------------------- ... ------------------ ------------------
5.0821668822067325 0.15718980192655577 ... 61.983917861125235  58.91792108166479
 5.084728396303883 0.14737015914787954 ...  57.88890282620754  59.00774236977636
 5.085264909636745  0.1389902386785043 ...  53.79388779128984  59.09756365788794
 5.083777223035887 0.13205028367278138 ...  49.69887275637214  59.18738494599951
 5.080263873779852 0.12654751559064142 ...  45.60385772145445  59.27720623411109
 5.074724938721787 0.12247556420484945 ...  41.50884268653676  59.36702752222266
 5.067162831007007 0.11982519429157105 ...  37.41382765161906  59.45684881033424
5.0575776955772405 0.11858353735920041 ... 

In [ ]:
#table.columns
table.info
#table['nw_x'], table['nw_y']
#table_sel = table['nw_x', 'nw_y']
#table_sel

In [ ]:
camera = LsstCam().getCamera()

In [ ]:
#camera

ORGINAL for Rubin TV

In [ ]:
# butler = Butler(
#     "embargo", 
#     # collections=["LSSTComCam/quickLook"], 
#     collections=["LSSTComCam/nightlyValidation"], 
#     instrument="LSSTComCam"
# )
# camera = LsstComCam().getCamera()

In [ ]:
# dayObs = 20241106 #20241029 #20241028
# seqNum = 251      #33       #10
# visit_id = int(f"{dayObs}{seqNum:05d}")

In [ ]:
# icSrcs = {}
# for i in range(9):
#     icSrcs[i] = butler.get("icSrc", dataId={"visit":visit_id, "detector":i})
# visitInfo = butler.get("postISRCCD.visitInfo", dataId={"exposure":visit_id, "detector":4})
# table = makeTableFromSourceCatalogs(icSrcs, visitInfo)

In [ ]:
#fa figura e tabella
fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(10, 9), sharex=True, sharey=True)
makeAzElPlot(fig, axes, table, camera,saveTableAs="ellipticity_wJosh_imsim145_ccd90.csv")
plt.show()

In [ ]:
# fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(10, 9), sharex=True, sharey=True)
# makeFocalPlanePlot(fig, axes, table, camera)
# plt.show()

In [341]:
# fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(10, 9), sharex=True, sharey=True)
# makeEquatorialPlot(fig, axes, table, camera)
# plt.show()